In [2]:
from typing import (Tuple,List,Dict,Union)
import pandas as pd
import numpy as np

from torch.nn import (Tanh,ReLU)
import torch

In [ ]:
def dynamics(y_t,sigma):
    
    y_next = torch.zeros(2)
    y_next[0] = 1.0 * torch.tanh(y_t[0]) + 0.8 * y_t[0]*y_t[1] + 1.0*y_t[1] - 1.0 * ReLU()

In [3]:
T_total = 2000
y = np.zeros((2, T_total))
y[:,0] = np.array([0.5, 0.5])

In [10]:
torch.zeros(2)

tensor([0., 0.])

In [12]:
torch.min(torch.tensor([1,2]))

tensor(1)